In [130]:
import pathlib
import os
from fastai.vision.all import ImageDataLoaders, Resize, imagenet_stats, vision_learner, resnet34, error_rate
from fastai.vision.data import get_image_files
import numpy as np

from fastai.vision.all import RandomResizedCrop, Flip, Dihedral, ToTensor, Normalize

# Ensure the path is correct and points to the directory containing images
chrono_images_path = pathlib.Path('/home/ubuntu/chrono-analyzer/data/')

# Get all image files from the directory
fnames = get_image_files(chrono_images_path)




